In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My\ Drive/Legal\ DS/BERTSpanCRF/rhetorical-role-baseline/

In [ ]:
!pip install -r requirements_colab.txt

In [ ]:
!python tokenize_files.py 'ekstep_train_seg_ind.json' 'ekstep_dev_seg_ind.json' 'ekstep_dev_seg_ind.json'

In [ ]:
!python baseline_run.py

In [ ]:
import time
import gc
from datetime import datetime
from os import makedirs

import torch

from eval_run import eval_and_save_metrics
from utils import get_device, ResultWriter, log
from task import pubmed_task
from train import SentenceClassificationTrainer
from models import BertHSLN
import os

BERT_MODEL = "bert-base-uncased"

config = {
    "bert_model": BERT_MODEL,
    "bert_trainable": False,
    "model": BertHSLN.__name__,
    "cacheable_tasks": [],

    "dropout": 0.5,
    "word_lstm_hs": 758,
    "att_pooling_dim_ctx": 200,
    "att_pooling_num_ctx": 15,

    "lr": 3e-05,
    "lr_epoch_decay": 0.9,
    "batch_size":  32,
    "max_seq_length": 128,
    "max_epochs": 20,
    "early_stopping": 5,
    "span_crf": True,
    "crf": False,

}

In [ ]:
from allennlp.data.dataset_readers.dataset_utils import enumerate_spans
import torch

In [ ]:
def get_span_indices(sentence_mask):
  all_span_ids = []
  for each in range(len(sentence_mask)):
    each_span_ids = enumerate_spans(sentence_mask[each][(sentence_mask[each].nonzero())])
    print("Each ", each_span_ids)
    all_span_ids.append(each_span_ids)

  print("all ", all_span_ids)
  max_span_len = max(len(x) for x in all_span_ids)
  span_ids = [x+[[0,0]]*(max_span_len-len(x)) for x in all_span_ids]
  span_indices = torch.tensor(span_ids)
  return span_indices

In [ ]:
s_mask = torch.tensor([[1,1,1,1],[1,0,0,0]])
s_mask1 = torch.tensor([1,1,1,1])
s_indices = get_span_indices(s_mask)
s_indices

In [ ]:
s_indices [0]

In [ ]:
def get_spans(sentence_mask):
  all_span_ids = []
  each_span_ids = enumerate_spans(sentence_mask.nonzero())
    #print("Each ", each_span_ids)
  all_span_ids.append(each_span_ids)
  
  return all_span_ids

In [ ]:
get_spans(s_mask1)